Please check this [article](https://medium.com/analytics-vidhya/an-introduction-to-multi-label-text-classification-b1bcb7c7364c?sk=8a30075009552cfd4a7534663edaed7e) for detailed explanation.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud,STOPWORDS

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score
from sklearn.metrics import hamming_loss
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import MultinomialNB
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import LabelPowerset

In [ ]:
train_data = pd.read_csv('/kaggle/input/topic-modeling-for-research-articles/train.csv')
test_data = pd.read_csv('/kaggle/input/topic-modeling-for-research-articles/test.csv')
train_data.head()


In [ ]:
test_data.head()

In [ ]:
print(train_data.shape)
print(test_data.shape)

In [ ]:
x=train_data.iloc[:,3:].sum()
rowsums=train_data.iloc[:,2:].sum(axis=1)
no_label_count = 0
for sum in rowsums.items():
    if sum==0:
        no_label_count +=1

print("Total number of articles = ",len(train_data))
print("Total number of articles without label = ",no_label_count)
print("Total labels = ",x.sum())

In [ ]:
print("Check for missing values in Train dataset")
print(train_data.isnull().sum().sum())
print("Check for missing values in Test dataset")
null_check=test_data.isnull().sum()
print(null_check)

In [ ]:
x=train_data.iloc[:,3:].sum()
#plot
plt.figure(figsize=(12,12))
ax= sns.barplot(x.index, x.values, alpha=0.8)
plt.title("Class counts")
plt.ylabel('# of Occurrences', fontsize=12)
plt.xlabel('Label ', fontsize=12)

rects = ax.patches
labels = x.values
for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2, height + 5, label, ha='center', va='bottom')

plt.show()


As the dataset is imbalanced, MLSMOTE technique can be used for sampling. As of now, I am not using any sampling techniques.

In [ ]:
x=rowsums.value_counts()

#plot
plt.figure(figsize=(12,12))
ax = sns.barplot(x.index, x.values, alpha=0.8)
plt.title("Multiple tags per article")
plt.ylabel('# of Occurrences', fontsize=12)
plt.xlabel('# of Labels ', fontsize=12)

#adding the text labels
rects = ax.patches
labels = x.values
for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2, height + 5, label, ha='center', va='bottom')

plt.show()


In [ ]:
train_data['Text']=train_data['TITLE']+' '+train_data['ABSTRACT']

In [ ]:
train_data.drop(columns=['TITLE','ABSTRACT'], inplace=True)
train_data.head(10)

In [ ]:
plt.figure(figsize=(12,12))
#text = description_category.description.values
cloud = WordCloud(stopwords=STOPWORDS, background_color='black', collocations=False, width=2500, height=1800).generate(" ".join(train_data['Text']))
plt.axis('off')
plt.imshow(cloud)

In [ ]:
train_data['Text'][5]

We have to remove symbols and punctuations

In [ ]:
#Remove Stopwords
stop_words = set(stopwords.words('english'))

# function to remove stopwords
def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

train_data['Text'] = train_data['Text'].apply(lambda x: remove_stopwords(x))

In [ ]:

stemmer = SnowballStemmer("english")
def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence

train_data['Text'] = train_data['Text'].apply(stemming)

In [ ]:
train_data['Text'][5]

In [ ]:
categories=['Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance'] 
train_data[categories].head()

In [ ]:
#split the data

x_train, x_test, y_train, y_test = train_test_split(train_data['Text'], train_data[categories], test_size=0.2, random_state=40, shuffle=True)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)



# MultiLabel Classification

# Binary Relevace

In [ ]:
# using binary relevance



pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', BinaryRelevance(MultinomialNB())),
            ])

# train
pipeline.fit(x_train, y_train)

# predict
predictions = pipeline.predict(x_test)


from sklearn.metrics import accuracy_score
print('Accuracy = ', accuracy_score(y_test,predictions))
print('F1 score is ',f1_score(y_test, predictions, average="micro"))
print('Hamming Loss is ', hamming_loss(y_test, predictions))

In [ ]:
# using binary relevance with Logistic Regression

pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', BinaryRelevance(LogisticRegression(solver='sag'))),
            ])

# train
pipeline.fit(x_train, y_train)

# predict
predictions = pipeline.predict(x_test)


from sklearn.metrics import accuracy_score
print('Accuracy = ', accuracy_score(y_test,predictions))
print('F1 score is ',f1_score(y_test, predictions, average="micro"))
print('Hamming Loss is ', hamming_loss(y_test, predictions))

# Classifier Chains

In [ ]:
# using classifier chains with MultinomialNB


# initialize classifier chains multi-label classifier
# with a gaussian naive bayes base classifier
pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', ClassifierChain(MultinomialNB())),
            ])
# ClassifierChain(GaussianNB())

# train
pipeline.fit(x_train, y_train)

# predict
predictions = pipeline.predict(x_test)

print('Accuracy = ', accuracy_score(y_test,predictions))
print('F1 score is ',f1_score(y_test, predictions, average="micro"))
print('Hamming Loss is ', hamming_loss(y_test, predictions))

In [ ]:
pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', ClassifierChain(LogisticRegression(solver='sag'))),
            ])
# ClassifierChain(GaussianNB())

# train
pipeline.fit(x_train, y_train)

# predict
predictions = pipeline.predict(x_test)

print('Accuracy = ', accuracy_score(y_test,predictions))
print('F1 score is ',f1_score(y_test, predictions, average="micro"))
print('Hamming Loss is ', hamming_loss(y_test, predictions))

# Label Powerset

In [ ]:
# using Label Powerset

# initialize label powerset multi-label classifier
pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf',  LabelPowerset(LogisticRegression())),
            ])
#classifier = LabelPowerset(LogisticRegression())
# train
pipeline.fit(x_train, y_train)
# predict
predictions = pipeline.predict(x_test)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print('F1 score is ',f1_score(y_test, predictions, average="micro"))
print('Hamming Loss is ', hamming_loss(y_test, predictions))

# References

* https://www.analyticsvidhya.com/blog/2017/08/introduction-to-multi-label-classification/

* https://towardsdatascience.com/journey-to-the-center-of-multi-label-classification-384c40229bff